In [46]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [47]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,North Platte,US,2022-07-26 05:59:24,41.1239,-100.7654,292.10,97,38,3.13
1,1,Buluang,PH,2022-07-26 06:00:18,13.3068,123.3425,303.50,71,89,2.02
2,2,Fuerte Olimpo,PY,2022-07-26 06:00:19,-21.0415,-57.8738,295.98,41,1,2.42
3,3,Clyde River,CA,2022-07-26 06:00:19,70.4692,-68.5914,275.28,100,100,3.60
4,4,Mayumba,GA,2022-07-26 06:00:20,-3.4320,10.6554,294.51,90,100,1.32
5,5,Kasongo-Lunda,CD,2022-07-26 06:00:21,-6.4667,16.8167,293.85,49,5,0.72
6,6,Axim,GH,2022-07-26 06:00:21,4.8699,-2.2405,296.76,85,69,1.85
7,7,Beni Mazar,EG,2022-07-26 06:00:22,28.4962,30.8029,303.35,30,0,4.10
8,8,Kapaa,US,2022-07-26 06:00:23,22.0752,-159.3190,300.14,78,11,3.13
9,9,Cimislia,MD,2022-07-26 06:00:23,46.5200,28.7842,294.23,37,1,1.04


In [50]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 250
What is the maximum temperature you would like for your trip? 300


In [51]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,North Platte,US,2022-07-26 05:59:24,41.1239,-100.7654,292.10,97,38,3.13
2,2,Fuerte Olimpo,PY,2022-07-26 06:00:19,-21.0415,-57.8738,295.98,41,1,2.42
3,3,Clyde River,CA,2022-07-26 06:00:19,70.4692,-68.5914,275.28,100,100,3.60
4,4,Mayumba,GA,2022-07-26 06:00:20,-3.4320,10.6554,294.51,90,100,1.32
5,5,Kasongo-Lunda,CD,2022-07-26 06:00:21,-6.4667,16.8167,293.85,49,5,0.72
6,6,Axim,GH,2022-07-26 06:00:21,4.8699,-2.2405,296.76,85,69,1.85
9,9,Cimislia,MD,2022-07-26 06:00:23,46.5200,28.7842,294.23,37,1,1.04
10,10,Leningradskiy,RU,2022-07-26 06:00:24,69.3833,178.4167,274.54,91,100,5.16
12,12,Hobyo,SO,2022-07-26 06:00:25,5.3505,48.5268,299.97,62,89,9.67
13,13,Novyy Urengoy,RU,2022-07-26 06:00:25,66.0833,76.6333,290.91,42,64,6.13


In [52]:
preferred_cities_df.count()

City_ID       430
City          430
Country       427
Date          430
Lat           430
Lng           430
Max Temp      430
Humidity      430
Cloudiness    430
Wind Speed    430
dtype: int64

In [53]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,North Platte,US,292.10,41.1239,-100.7654,
2,Fuerte Olimpo,PY,295.98,-21.0415,-57.8738,
3,Clyde River,CA,275.28,70.4692,-68.5914,
4,Mayumba,GA,294.51,-3.4320,10.6554,
5,Kasongo-Lunda,CD,293.85,-6.4667,16.8167,
6,Axim,GH,296.76,4.8699,-2.2405,
9,Cimislia,MD,294.23,46.5200,28.7842,
10,Leningradskiy,RU,274.54,69.3833,178.4167,
12,Hobyo,SO,299.97,5.3505,48.5268,
13,Novyy Urengoy,RU,290.91,66.0833,76.6333,


In [54]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [58]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [59]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))